# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import operator
import time
from sklearn.preprocessing import Imputer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA    
from sklearn.preprocessing import LabelEncoder
!pip install mca
import mca
import chardet
# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [2]:
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';', dtype=str)
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';', dtype=str)

In [3]:
# Be sure to add in a lot more cells (both markdown and code) to document your
# approach and findings!

In [4]:
azdias.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9,0,NaN,NaN,NaN,NaN,21,11,...,4,8,11,10,3,9,4,5,2,1
2,910225,-1,9,17,NaN,NaN,NaN,NaN,17,10,...,2,9,9,6,3,9,2,5,2,3
3,910226,2,1,13,NaN,NaN,NaN,NaN,13,1,...,0,7,10,11,NaN,9,7,3,2,4
4,910241,-1,1,20,NaN,NaN,NaN,NaN,14,3,...,2,3,5,4,2,9,3,4,1,3


In [5]:
azdias["LNR"].nunique()

891221

It seems like each row has unique LNR value.

In [6]:
customers["LNR"].nunique() == customers.shape[0]

True

In [7]:
np.intersect1d(azdias["LNR"], customers["LNR"])

array([], dtype=object)

In [8]:
with open("dias.csv.csv", "rb") as dias:
    dias_content = dias.read()
    print(chardet.detect(dias_content))

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [9]:
dias = pd.read_csv("dias.csv.csv", encoding="Windows-1252")

In [10]:
dias.head(20)

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,NaN,NaN,0,no classification possible
2,NaN,NaN,1,passive elderly
3,NaN,NaN,2,cultural elderly
4,NaN,NaN,3,experience-driven elderly
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,NaN,NaN,1,< 30 years
7,NaN,NaN,2,30 - 45 years
8,NaN,NaN,3,46 - 60 years
9,NaN,NaN,4,> 60 years


In [11]:
dias = dias.fillna(method="ffill")

In [12]:
dias.head(20)

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,AGER_TYP,best-ager typology,0,no classification possible
2,AGER_TYP,best-ager typology,1,passive elderly
3,AGER_TYP,best-ager typology,2,cultural elderly
4,AGER_TYP,best-ager typology,3,experience-driven elderly
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,ALTERSKATEGORIE_GROB,age classification through prename analysis,1,< 30 years
7,ALTERSKATEGORIE_GROB,age classification through prename analysis,2,30 - 45 years
8,ALTERSKATEGORIE_GROB,age classification through prename analysis,3,46 - 60 years
9,ALTERSKATEGORIE_GROB,age classification through prename analysis,4,> 60 years


In [13]:
attribute_names = dias[["Attribute", "Description"]].drop_duplicates()

In [14]:
attribute_desc = {}

In [15]:
for index, row in attribute_names.iterrows():
    attribute_desc[row[0]] = row[1]

In [16]:
attribute_desc

{'AGER_TYP': 'best-ager typology',
 'ALTERSKATEGORIE_GROB': 'age classification through prename analysis ',
 'ALTER_HH': 'main age within the household',
 'ANREDE_KZ': 'gender',
 'ANZ_HAUSHALTE_AKTIV': 'number of households in the building',
 'ANZ_HH_TITEL': 'number of academic title holder in building',
 'ANZ_PERSONEN': 'number of adult persons in the household',
 'ANZ_TITEL': 'number of professional title holder in household ',
 'BALLRAUM': 'distance to next urban centre ',
 'BIP_FLAG': 'business-flag indicating companies in the building',
 'CAMEO_DEUG_2015': 'CAMEO classification 2015 - Uppergroup',
 'CAMEO_DEU_2015': 'CAMEO classification 2015 - detailled classification',
 'CAMEO_DEUINTL_2015': '(each German CAMEO code belongs to one international code)',
 'CJT_GESAMTTYP': 'customer journey typology',
 'D19_BANKEN_ANZ_12': 'transaction activity BANKS in the last 12 months',
 'D19_BANKEN_ANZ_24': 'transaction activity BANKS in the last 24 months',
 'D19_BANKEN_DATUM': 'actuality of 

In [17]:
azdias_clean = azdias.rename(columns=attribute_desc)

In [18]:
customers_clean = customers.rename(columns=attribute_desc)

In [19]:
azdias_clean.columns

Index(['LNR', 'best-ager typology', 'AKT_DAT_KL',
       'main age within the household', 'ALTER_KIND1', 'ALTER_KIND2',
       'ALTER_KIND3', 'ALTER_KIND4', 'ALTERSKATEGORIE_FEIN',
       'number of households in the building',
       ...
       'VHN', 'VK_DHT4A', 'VK_DISTANZ', 'VK_ZG11',
       'likelihood of a child present in this household',
       'length of residence', 'residential-area ',
       'typification of energy consumers', 'gender',
       'age classification through prename analysis '],
      dtype='object', length=366)

In [20]:
customers_clean.columns

Index(['LNR', 'best-ager typology', 'AKT_DAT_KL',
       'main age within the household', 'ALTER_KIND1', 'ALTER_KIND2',
       'ALTER_KIND3', 'ALTER_KIND4', 'ALTERSKATEGORIE_FEIN',
       'number of households in the building',
       ...
       'VK_ZG11', 'likelihood of a child present in this household',
       'length of residence', 'residential-area ',
       'typification of energy consumers', 'PRODUCT_GROUP', 'CUSTOMER_GROUP',
       'ONLINE_PURCHASE', 'gender',
       'age classification through prename analysis '],
      dtype='object', length=369)

In [21]:
customers_clean.CUSTOMER_GROUP.head()

0     MULTI_BUYER
1    SINGLE_BUYER
2     MULTI_BUYER
3     MULTI_BUYER
4     MULTI_BUYER
Name: CUSTOMER_GROUP, dtype: object

In [22]:
customers_clean.ONLINE_PURCHASE.value_counts()

0    174356
1     17296
Name: ONLINE_PURCHASE, dtype: int64

In [23]:
customers_clean.PRODUCT_GROUP.unique()

array(['COSMETIC_AND_FOOD', 'FOOD', 'COSMETIC'], dtype=object)

In [24]:
list(customers_clean.columns)

['LNR',
 'best-ager typology',
 'AKT_DAT_KL',
 'main age within the household',
 'ALTER_KIND1',
 'ALTER_KIND2',
 'ALTER_KIND3',
 'ALTER_KIND4',
 'ALTERSKATEGORIE_FEIN',
 'number of households in the building',
 'number of academic title holder in building',
 'ANZ_KINDER',
 'number of adult persons in the household',
 'ANZ_STATISTISCHE_HAUSHALTE',
 'number of professional title holder in household ',
 'ARBEIT',
 'distance to next urban centre ',
 'CAMEO classification 2015 - detailled classification',
 'CAMEO classification 2015 - Uppergroup',
 'CAMEO_INTL_2015',
 'customer journey typology',
 'CJT_KATALOGNUTZER',
 'CJT_TYP_1',
 'CJT_TYP_2',
 'CJT_TYP_3',
 'CJT_TYP_4',
 'CJT_TYP_5',
 'CJT_TYP_6',
 'transaction activity BANKS in the last 12 months',
 'transaction activity BANKS in the last 24 months',
 'actuality of the last transaction for the segment banks TOTAL',
 'D19_BANKEN_DIREKT',
 'D19_BANKEN_GROSS',
 'D19_BANKEN_LOKAL',
 'actuality of the last transaction for the segment banks O

In [25]:
dias.head(1)    

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown


In [26]:
customers_clean = customers_clean_backup.copy()

NameError: name 'customers_clean_backup' is not defined

In [ ]:
# for index, row in dias.iterrows():
#     attr, desc, value, meaning = row
#     if desc in list(customers_clean.columns):        
#         customers_clean.loc[customers_clean[desc] == value, desc] = meaning

In [ ]:
customers_clean = pd.get_dummies(customers_clean.iloc[:, 1:], dummy_na=True)

In [ ]:
list(customers_clean.columns)

In [ ]:
len(list(customers_clean.columns))

In [ ]:
azdias_clean = pd.get_dummies(azdias_clean.iloc[:, 1:], dummy_na=True)

In [ ]:
len(list(azdias_clean.columns))

In [ ]:
list(azdias_clean.columns)

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')